# Robot Interface Example

## C++ Driver

This notebook shows how to connect to the robot and send joint velocity/torque commands to it. Before running the notebook, make sure you enable the FCI interface and run the C++ fr3_bridge as follows:

**Joint Velocity Interface**
```bash
fr3_joint_interface 10.42.0.4 franka velocity
```

**Joint Torque Interface**
```bash
fr3_joint_interface 10.42.0.4 franka torque
```

**Note**: If the robot is connected to another computer, make sure that the two computers are on the same local network. Also use the `tools/multicast_config.py` tool to enable the multicast on both computers:

```bash
sudo tools/multicast_config.py <interface name>
```
where interface name is network interface that is on the same local network with the computer connected to the robot. (The computer running the real-time kernel)

## Python Bridge

Now that the interface is up, we can communicate with the robot through Python. Let's instantiate the communication manager:

In [4]:
import sys 
from FR3Py.robot.interface import FR3RobotInterface
robot = FR3RobotInterface()

Interface Running...


If the interface the successfully exchanging data between the robot and the Python side, we should be able to read the state of the robot. Otherwise, we will get a None.

In [5]:
robot.getStates()

Now that we successfully read the state, let's use a simple Jacobian pseudo inverse controller to command the robot's end-effector along the cartesian Z axis:

In [6]:
import time
import numpy as np
from FR3Py.controllers.jacobianPseudoInv import WaypointController

controller = WaypointController(kp=1.5)
# Read the initial state of the robot
state = robot.getStates()
if state is not None:
    q, dq = state['q'], state['dq']
    p0 = controller.robot.getInfo(q,dq)['P_EE']
    R0 = controller.robot.getInfo(q,dq)['R_EE']
    T0 = np.vstack([np.hstack([R0, p0.reshape(3,1)]), np.array([0,0,0,1])])
    
    start_time = time.time()
    while True:
        time.sleep(0.01)
        t = time.time() - start_time
        x = 0.1 * np.sin(t)
        T= T0 @ np.vstack([np.hstack([np.eye(3), np.array([0,0,x]).reshape(3,1)]), np.array([0,0,0,1])])
        state = robot.get_state()
        q, dq = state['q'], state['dq']
        cmd = controller.compute(q, dq, T_cmd=T)
        robot.send_joint_command(cmd[0:7])
else:
    print('State is None. check the connection')

State is None. check the connection
